# Test TF datasets features

In [14]:
import tensorflow as tf
import pandas as pd
import numpy as np
from IPython.display import display

In [64]:
X['a']*2

0     0
1     2
2     4
3     6
4     8
5    10
6    12
7    14
8    16
9    18
Name: a, dtype: int64

In [68]:
n = 10
X = pd.DataFrame([[i] for i in range(n)], columns=['a'])
# X['b'] = np.random.randn(n)*0.1 + 4

y = pd.DataFrame(X['a']*2).rename({'a': 'label'}, axis=1) - 1
desc = pd.DataFrame([f"Value {i}" for i in range(1, n+1)], columns=['desc'])

display(X, y, desc)

,a
0,0
1,1
2,2
3,3
4,4
5,5
6,6
7,7
8,8
9,9


,label
0,-1
1,1
2,3
3,5
4,7
5,9
6,11
7,13
8,15
9,17


,desc
0,Value 1
1,Value 2
2,Value 3
3,Value 4
4,Value 5
5,Value 6
6,Value 7
7,Value 8
8,Value 9
9,Value 10


In [112]:
ds = tf.data.Dataset.from_tensor_slices((X, y))
ds_with_labels = tf.data.Dataset.zip((ds,
                                      tf.data.Dataset.from_tensor_slices((desc, )))).batch(4)
ds = ds.shuffle(n).batch(4).prefetch(buffer_size=4)

In [113]:
for batch in ds_with_labels:
    print(batch)

((<tf.Tensor: shape=(4, 1), dtype=int64, numpy=
array([[0],
       [1],
       [2],
       [3]])>, <tf.Tensor: shape=(4, 1), dtype=int64, numpy=
array([[-1],
       [ 1],
       [ 3],
       [ 5]])>), (<tf.Tensor: shape=(4, 1), dtype=string, numpy=
array([[b'Value 1'],
       [b'Value 2'],
       [b'Value 3'],
       [b'Value 4']], dtype=object)>,))
((<tf.Tensor: shape=(4, 1), dtype=int64, numpy=
array([[4],
       [5],
       [6],
       [7]])>, <tf.Tensor: shape=(4, 1), dtype=int64, numpy=
array([[ 7],
       [ 9],
       [11],
       [13]])>), (<tf.Tensor: shape=(4, 1), dtype=string, numpy=
array([[b'Value 5'],
       [b'Value 6'],
       [b'Value 7'],
       [b'Value 8']], dtype=object)>,))
((<tf.Tensor: shape=(2, 1), dtype=int64, numpy=
array([[8],
       [9]])>, <tf.Tensor: shape=(2, 1), dtype=int64, numpy=
array([[15],
       [17]])>), (<tf.Tensor: shape=(2, 1), dtype=string, numpy=
array([[b'Value 9'],
       [b'Value 10']], dtype=object)>,))


In [101]:
for batch in ds:
    print(batch)

(<tf.Tensor: shape=(4, 1), dtype=int64, numpy=
array([[8],
       [9],
       [7],
       [5]])>, <tf.Tensor: shape=(4, 1), dtype=int64, numpy=
array([[15],
       [17],
       [13],
       [ 9]])>)
(<tf.Tensor: shape=(4, 1), dtype=int64, numpy=
array([[3],
       [2],
       [1],
       [6]])>, <tf.Tensor: shape=(4, 1), dtype=int64, numpy=
array([[ 5],
       [ 3],
       [ 1],
       [11]])>)
(<tf.Tensor: shape=(2, 1), dtype=int64, numpy=
array([[4],
       [0]])>, <tf.Tensor: shape=(2, 1), dtype=int64, numpy=
array([[ 7],
       [-1]])>)


In [103]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(5, activation='relu', input_shape=(1, )),
    tf.keras.layers.Dense(3, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [104]:
model.compile(loss=tf.keras.losses.MAE,
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              metrics=['accuracy'])

In [105]:
model.fit(ds, epochs=50)

Epoch 1/50
3/3 [==============================] - 1s 7ms/step - loss: 8.8569 - accuracy: 0.0000e+00
Epoch 2/50
3/3 [==============================] - 0s 6ms/step - loss: 8.4032 - accuracy: 0.0000e+00
Epoch 3/50
3/3 [==============================] - 0s 7ms/step - loss: 8.0943 - accuracy: 0.0000e+00
Epoch 4/50
3/3 [==============================] - 0s 7ms/step - loss: 7.7847 - accuracy: 0.0000e+00
Epoch 5/50
3/3 [==============================] - 0s 9ms/step - loss: 7.5028 - accuracy: 0.0000e+00
Epoch 6/50
3/3 [==============================] - 0s 8ms/step - loss: 7.2110 - accuracy: 0.1000
Epoch 7/50
3/3 [==============================] - 0s 8ms/step - loss: 6.8483 - accuracy: 0.1000
Epoch 8/50
3/3 [==============================] - 0s 7ms/step - loss: 6.5076 - accuracy: 0.1000
Epoch 9/50
3/3 [==============================] - 0s 9ms/step - loss: 6.0893 - accuracy: 0.1000
Epoch 10/50
3/3 [==============================] - 0s 8ms/step - loss: 5.6388 - accuracy: 0.1000
Epoch 11/50
3/3 [==

In [106]:
model.predict([[5]])

array([[9.230188]], dtype=float32)

In [119]:
for val in ds_with_labels:
    ((X, y), (labels, )) = val
    print(model.predict(X), y.numpy(), labels.numpy())

[[-0.00680409]
 [ 1.3416592 ]
 [ 3.3237505 ]
 [ 5.292563  ]] [[-1]
 [ 1]
 [ 3]
 [ 5]] [[b'Value 1']
 [b'Value 2']
 [b'Value 3']
 [b'Value 4']]
[[ 7.261376]
 [ 9.230188]
 [11.199   ]
 [13.167812]] [[ 7]
 [ 9]
 [11]
 [13]] [[b'Value 5']
 [b'Value 6']
 [b'Value 7']
 [b'Value 8']]
[[15.136626]
 [17.10544 ]] [[15]
 [17]] [[b'Value 9']
 [b'Value 10']]
